# RidgeSketch : Tutorial to add a new sketching technique

    RidgeSketch package
    Authors : Nidham Gazagnadou, Robert Gower, Mark Ibrahim
   
The aim of this project is to provide an open source package in Python for solving large scale ridge regression using the sketch-and-project technique.

This tutorial gives an overview of :
- add a new skething technique
- check that tests are passed
- benchmark it against other sketching methods

It is shown how to add a new sketching method a user would like to try and to compare its performance against pre-existing sketching methods or other solvers (like CG or direct solver).

As an example, we will show how to add the **partitioning sketch** which divides the matrix $A$ in blocks
$A_1, A_2, ..., A_{\text{n_folds}}$ each of size `sketch_size` and samples them cyclicaly before resetting randomly new blocks.


It is also shown how to create a **new benchmark** in order to compare the performance of different sketching methods over different settings (e.g. with or without Kernel, different sketching size etc).

### Table of content

[0. Prerequisites](#prerequisites)<br>
[1. Load data](#load_data)<br>
[2. Create the new sketch in *sketching.py*](#sketching)<br>
[3. Add to solvers in *ridge_sketch.py*](#add_to_solvers)<br>
[4. Write tests](#tests)<br>
[5. Write a benchmark](#benchmark)<br>

<a id='prerequisites'></a>
## 0. Prerequisites

In [ ]:
import sys
sys.path.append("../")

In [ ]:
# Make sure the 'ridgesketch-env' environment is activated
# and that all requirements are installed with
# $pip install -r requirements.txt

In [ ]:
# %matplotlib inline

In [ ]:
import numpy as np
from ridge_sketch import RidgeSketch

<a id='load_data'></a>
## 1. Load data

In [ ]:
np.random.seed(0)

# Generating a dataset
n_samples, n_features = 10, 500
X = np.random.rand(n_samples, n_features)
y = np.random.rand(n_samples, 1) # Warning: y should be of size (n_samples, 1) not (n_samples, )
print(f"X shape {(n_samples, n_features)}, y shape {y.shape}")

<a id='sketching'></a>
## 2. Create the new sketch in *sketching.py*

Create a new sketching class inheriting from the `Sketch` class in *sketching.py*. It must have 4 methods

- `__init__`: initialize required attributes (they already contain `A`, `b`, sketch_size, and the number of rows of A in `m`) which usually consist of the sketch matrix `S` or an equivalent representation (cf `SubsampleSketch`, `CountSketch` and `SubcountSketch`)
- `set_sketch`: method generating the sketch matrix `S` or its representation
- `sketch`: a method taking as input the current residual and returns `SA`=$S^\top A$, `SAS`=$S^\top A S$ and the sketched residual `rs`=$r_S = S^\top r$ (cf algorithm ? in the paper)
- `udate_iterate`: updates the weights using the least norm solution `lmbda`

Example: the subsample sketching method

```python
class SubsampleSketch(Sketch):
    def __init__(self, A, b, sketch_size):
        super().__init__(A, b, sketch_size)
        self.sample_indices = None

    def set_sketch(self):
        """Generates subsampling indices"""
        self.sample_indices = generate_sample_indices(self.m, self.sketch_size)

    def sketch(self, r):
        self.set_sketch()
        SA = self.A.get_rows(self.sample_indices)
        SAS = SA[:, self.sample_indices]
        rs = r[self.sample_indices]
        return SA, SAS, rs

    def update_iterate(self, w, lmbda, step_size=1.0):
        if self.sample_indices is None:
            raise AttributeError("sample indices before updating iterate")
        w[self.sample_indices] -= step_size * lmbda
        return w
```

In [ ]:
class PartitionSketch(Sketch):
    def __init__(self, A, b, sketch_size):
        super().__init__(A, b, sketch_size)
        self.iter = 0 # number of iteration modulo the number of folds
        self.n_folds = int(np.floor(self.m / self.sketch_size))
        self.folds = None # list of the arrays of indices
        # `self.sample_indices` need to be a list for A.get_rows() to work
        self.sample_indices = None # list of selected indices

    def set_sketch(self):
        """
        Selects one of the row partitions of A.

        Generates random arrays of size `skecth_dim`
        containing subsampling indices if not already
        done.
        Some rows are dropped to build subsets of size
        `sketch_size`.
        """
        if self.folds is None or self.iter == 0:
            idx = np.arange(self.m)
            np.random.shuffle(idx)
            idx = idx[:-(self.m % self.sketch_size)].copy()
            self.folds = np.array_split(idx, self.n_folds)
            # print(f"Creating {self.n_folds} folds, each of which of size {self.sketch_size}")

        # Going through all folds once before resampling new ones:
        self.sample_indices = self.folds[self.iter].tolist()
        # Sampling randomly a fold:
        # self.sample_indices = self.folds[np.random.randint(0, self.n_folds)].tolist()

    def sketch(self, r):
        self.set_sketch()
        self.iter += 1
        if self.iter % self.n_folds == 0:
            self.iter = 0
        SA = self.A.get_rows(self.sample_indices)
        SAS = SA[:, self.sample_indices]
        rs = r[self.sample_indices]
        return SA, SAS, rs

    def update_iterate(self, w, lmbda, step_size=1.0):
        if self.folds is None or self.sample_indices is None:
            raise AttributeError("set the partition and select one of them first")
        w[self.sample_indices] -= step_size * lmbda
        return w

Something you should be careful about is the type of `A`. As presented in `a_matrix.py`, it can be either a `AMatrix` object or a `AOperator` for which $A$ is not built. For both class, common operations are available like matrix-vector multiplication, `get_elements` or `get_rows` methods.

<a id='add_to_solvers'></a>
## 3. Add to solvers in *ridge_sketch.py*

Add the new solver to the list in the `ridge_sketch.py` on line 21.

WARNING: remove the "sketch" prefix.

In [1]:
SKETCH_SOLVERS = {
    "direct",
    "cg",
    "subsample",
    "coordinate descent",
    "gaussian",
    "count",
    "subcount",
    "hadamard",
    "partition",
    }

<a id='tests'></a>
## 4. Write tests

### 1) General tests
First, open `test/conftest.py` and add the name of your sketching method in the `ALL_SOLVERS` and `SKETCH_METHODS` lists. For instance,
- add `"partition",` at the end of `ALL_SOLVERS` one lines 30
- add `"PartitionSketch",` at the end of `SKETCH_METHODS` one line 46

In [ ]:
ALL_SOLVERS = [
    "direct",
    "cg",
    "subsample",
    "coordinate descent",
    "gaussian",
    "count",
    "subcount",
    "hadamard",
    "partition",
    ]

ALL_SOLVERS_EXCEPT_DIRECT = ALL_SOLVERS[1:]

SKETCH_SOLVERS = ALL_SOLVERS_EXCEPT_DIRECT[1:]

SKETCH_METHODS = [
    "SubsampleSketch",
    "CoordinateDescentSketch",
    "GaussianSketch",
    "CountSketch",
    "SubcountSketch",
    "HadamardSketch",
    "PartitionSketch",
    ]

Thus, all tests related to the vanilla, momentum, accelerated, kernel, etc versions of Ridge Sketch are also launched on the new sketch.

Slow tests are be launched on the new sketch too by running the following command
```bash
$ pytest . -v --runslow
```
where the `-v` is used to verbose the run tests.

**WARNING: the tests `test_all_sketch_solvers_solutions_sparse` and `test_all_sketch_solvers_operator_mode` are not passed by the sketching method presented in this notebook.**

### 2) Additional specific tests
Then, you can implement specific tests at the end of the the file `test/test_sketching.py` like this:

In [ ]:
class TestPartion:
    def test_partitions_number(self, A, b, sketch_size):
        """
        Verifies that we divide the row indices of A into folds of size
        `sketch_size`
        """
        partition = sketching.PartitionSketch(A, b, sketch_size)
        partition.set_sketch()
        m = A.shape[0]
        n_folds = int(np.floor(m / sketch_size))
        assert len(partition.folds) == n_folds

    def test_partitions_size(self, A, b, sketch_size):
        """Verifies that the folds are of size `sketch_size`"""
        partition = sketching.PartitionSketch(A, b, sketch_size)
        partition.set_sketch()
        assert len(partition.sample_indices) == sketch_size

### 3) Run the tests

Run
```bash
$ pytest . -v
```
add `--runslow` to run tests marked as slow.

<a id='benchmark'></a>
## 5. Write a benchmark

Open `benchmark_configs.py` and create a new benchmark at the end like this:

In [ ]:
# Partition Benchmarks
partition_bench = Configs(name="partition")
partition_bench.problems = ("primal_random",) # primal or dual generated problem 
partition_bench.algo_modes = ("auto",) # only vanilla Ridge Sketch method
partition_bench.operator_modes = (False,) # only matrix mode
partition_bench.solvers = {"partition", "direct", "cg"} # solvers to compare with
partition_bench.smaller_dimension = 2000
partition_bench.larger_dimension = 8000

Then run 
```bash
$ python benchmarks.py partition_bench
```
and the results should be saved in the `benchmark_results/partition_bench` folder.